In [1]:
import oandapyV20
from oandapyV20 import API
from colorama import Fore
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.instruments as instruments

from dotenv import load_dotenv
import pandas as pd
from datetime import datetime, timedelta
from termcolor import colored
import os
import json
load_dotenv()

True

In [2]:
accountID = os.getenv("OANDA_ACCOUNT_ID")
api = API(access_token=os.getenv("OANDA_ACCESS_TOKEN"))


In [3]:
def process_streaming_response(response, temp_list):
    bid = float(response['closeoutBid'])
    ask = float(response['closeoutAsk'])
    mid = round((bid + ask) / 2, 3)
    temp_list.append(mid)


In [4]:
def get_candlestick_data(timeframe, temp_list):
    open = round(temp_list[0], 3)
    high = round(max(temp_list), 3)
    low = round(min(temp_list), 3)
    close = round(temp_list[-1], 3)
    
    data = {'Open': open, 'High': high, 'Low': low, 'Close': close}
    df = pd.DataFrame(data, index=[timeframe])
    
    return df

In [5]:
def calculate_indicators(df):
    
    # SMA
    df["SMA"] = df["Close"].rolling(window=5).mean()

    # RSI
    delta = df["Close"].diff()
    gain = (delta.where(delta > 0, 0)).ewm(span=5).mean()
    loss = (-delta.where(delta < 0, 0)).ewm(span=5).mean()
    rs = gain / loss
    df["RSI"] = 100 - (100 / (1 + rs))

    # MACD
    df["MACD"] = df["Close"].ewm(span=5).mean() - df["Close"].ewm(span=13).mean()

    # Stochastic Oscillator
    low_5, high_5 = df["Low"].rolling(window=5).min(), df["High"].rolling(window=5).max()
    df["%K"] = 100 * (df["Close"] - low_5) / (high_5 - low_5)
    df["%D"] = df["%K"].rolling(window=3).mean()
    
    return df

In [13]:
client = oandapyV20.API(access_token=os.getenv("OANDA_ACCESS_TOKEN"))

params = {'granularity': 'M1', 'count': 500}
r = instruments.InstrumentsCandles(instrument="USD_JPY",
                                   params=params)
client.request(r)

data = [{'Time': d['time'], # time is in UTC by default
         'High': d['mid']['h'], 
         'Close': d['mid']['c'], 
         'Low': d['mid']['l'], 
         'Open': d['mid']['o']} 
        for d in r.response['candles']]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df['Time'] = pd.to_datetime(df['Time']).dt.tz_convert('Asia/Singapore')
df.set_index('Time', inplace=True)
df[['High', 'Close', 'Low', 'Open']] = df[['High', 'Close', 'Low', 'Open']].apply(pd.to_numeric)

In [14]:
df = calculate_indicators(df)
df.dropna(inplace=True)

In [15]:
def streaming_data_pipeline(accountID, params, api, df):
    start_time = datetime.now()
    max_duration = timedelta(minutes=10)  # Set the duration for the time-based exit
    
    interval_start = datetime.now()
    interval = timedelta(minutes=1)  # interval for aggregating the candlestick data
    temp_list = []  # Initialize outside the loop to persist data across iterations
    
    r = pricing.PricingStream(accountID=accountID, params=params)
    
    try:
        rv = api.request(r)
        for tick in rv:
            # Check if the maximum duration has been exceeded
            if datetime.now() - start_time >= max_duration:
                print("Maximum duration reached, exiting...")
                break  # Exit the loop

            try:
                process_streaming_response(tick, temp_list)
                

                print(f"Time: {tick['time']}, {colored('closeoutBid:', 'green')} {tick['closeoutBid']}, {colored('closeoutAsk:', 'red')} {tick['closeoutAsk']}")
                print()
                if datetime.now() - interval_start >= interval:
                    interval_start = datetime.now()
                    if temp_list:
                        new_df = get_candlestick_data(interval_start, temp_list)
                        df = pd.concat([df, new_df])
                        temp_list.clear()
                        df = calculate_indicators(df)
                        print(df.tail(5))
                        print()
                        
            except Exception as e:
                print(colored("Processing heartbeat messages for network latency check", 'blue'))
                
    except oandapyV20.exceptions.V20Error as err:
        print(f"V20Error encountered: {err}")
    except KeyboardInterrupt:
        print("Streaming stopped by user.")
    finally:
        return df  # Return the DataFrame after exiting the loop

In [16]:
params = {'instruments': 'USD_JPY'}
streaming_data_pipeline(accountID, params, api, df)

Time: 2024-04-03T04:19:16.917635666Z, closeoutBid: 151.551, closeoutAsk: 151.570

Time: 2024-04-03T04:19:21.911602284Z, closeoutBid: 151.551, closeoutAsk: 151.569

Time: 2024-04-03T04:19:24.777461968Z, closeoutBid: 151.551, closeoutAsk: 151.570

Processing heartbeat messages for network latency check
Processing heartbeat messages for network latency check
Processing heartbeat messages for network latency check
Processing heartbeat messages for network latency check
Processing heartbeat messages for network latency check
Processing heartbeat messages for network latency check
Processing heartbeat messages for network latency check
Processing heartbeat messages for network latency check
Time: 2024-04-03T04:20:06.456029341Z, closeoutBid: 151.552, closeoutAsk: 151.571

Processing heartbeat messages for network latency check
Time: 2024-04-03T04:20:07.036247178Z, closeoutBid: 151.554, closeoutAsk: 151.572

Time: 2024-04-03T04:20:07.146420012Z, closeoutBid: 151.554, closeoutAsk: 151.573

Time

,High,Close,Low,Open,SMA,RSI,MACD,%K,%D
2024-04-03 03:55:00+08:00,151.564,151.564,151.561,151.562,NaN,NaN,0.000000,NaN,NaN
2024-04-03 03:56:00+08:00,151.567,151.566,151.564,151.566,NaN,100.000000,0.000123,NaN,NaN
2024-04-03 03:57:00+08:00,151.576,151.574,151.562,151.564,NaN,100.000000,0.000849,NaN,NaN
2024-04-03 03:58:00+08:00,151.574,151.562,151.559,151.574,NaN,34.146341,-0.000188,NaN,NaN
2024-04-03 03:59:00+08:00,151.572,151.568,151.561,151.561,151.5668,55.918367,0.000062,52.941176,NaN
...,...,...,...,...,...,...,...,...,...
2024-04-03 12:16:00+08:00,151.565,151.564,151.560,151.563,151.5512,92.473112,0.003988,96.774194,91.517324
2024-04-03 12:17:00+08:00,151.564,151.561,151.560,151.563,151.5554,72.290722,0.004586,87.096774,94.623656
2024-04-03 12:18:00+08:00,151.564,151.562,151.560,151.562,151.5598,75.017009,0.004899,88.000000,90.623656
2024-04-03 12:19:00+08:00,151.562,151.560,151.560,151.562,151.5622,57.920729,0.004464,66.666667,80.587814
